In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
pubchem_df= pd.read_csv('pubchem_data.csv')

In [12]:
pubchem_df.head()

pubchem_df.columns



pubchem_df.describe()

,cid,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,...,definedatomstereocnt,undefinedatomstereocnt,totalbondstereocnt,definedbondstereocnt,undefinedbondstereocnt,pclidcnt,gpidcnt,gpfamilycnt,annothitcnt,cidcdate
count,1.150900e+04,11509.000000,11509.000000,11509.000000,8806.000000,11509.000000,11509.000000,11509.000000,11509.000000,11509.000000,...,11509.000000,11509.000000,11509.000000,11509.000000,11509.000000,11509.000000,1.150900e+04,1.150900e+04,11509.000000,1.150900e+04
mean,3.717475e+07,538.457175,160.320057,886.103363,1.828401,36.810583,4.304805,8.796420,8.449474,537.999315,...,3.512990,1.502824,0.420627,0.312277,0.108350,3049.430620,9.005932e+03,3.146308e+03,5.872274,2.008917e+07
std,4.952511e+07,600.573729,256.885691,1447.804471,4.150668,41.453113,8.304964,11.317633,17.279320,600.364355,...,6.106369,4.105038,1.198740,1.040945,0.589702,14962.859368,4.936902e+04,2.770489e+04,5.782783,5.101057e+04
min,1.190000e+02,2.016000,0.000000,0.000000,-70.200000,0.000000,0.000000,0.000000,0.000000,2.015650,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,2.004092e+07
25%,4.472700e+05,295.370000,58.900000,340.000000,0.300000,20.000000,1.000000,4.000000,2.000000,295.096000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,2.005071e+07
50%,1.101262e+07,398.500000,96.200000,590.000000,2.400000,28.000000,2.000000,6.000000,5.000000,398.126000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000e+00,3.000000e+00,4.000000,2.006103e+07
75%,5.684333e+07,567.800000,178.000000,904.000000,4.000000,38.000000,4.000000,10.000000,9.000000,567.313000,...,5.000000,1.000000,0.000000,0.000000,0.000000,682.000000,1.020000e+03,3.340000e+02,10.000000,2.012032e+07
max,1.724189e+08,7595.000000,3260.000000,20600.000000,22.100000,484.000000,188.000000,215.000000,234.000000,7591.760000,...,153.000000,153.000000,14.000000,14.000000,9.000000,549746.000000,2.452859e+06,1.610091e+06,20.000000,2.024111e+07


In [14]:
pubchem_df.isnull().sum()

cid                          0
cmpdname                     0
cmpdsynonym               2352
mw                           0
mf                           0
polararea                    0
complexity                   0
xlogp                     2703
heavycnt                     0
hbonddonor                   0
hbondacc                     0
rotbonds                     0
inchi                        0
isosmiles                    0
canonicalsmiles              0
inchikey                     0
iupacname                  139
exactmass                    0
monoisotopicmass             0
charge                       0
covalentunitcnt              0
isotopeatomcnt               0
totalatomstereocnt           0
definedatomstereocnt         0
undefinedatomstereocnt       0
totalbondstereocnt           0
definedbondstereocnt         0
undefinedbondstereocnt       0
pclidcnt                     0
gpidcnt                      0
gpfamilycnt                  0
meshheadings              9004
annothit

In [24]:
# Drop columns that are unlikely to be useful for the ML model
columns_to_drop = ['cmpdsynonym', 'iupacname', 'meshheadings', 'annotation', 'annothits', 'aids']
pubchem_df_cleaned = pubchem_df.drop(columns=columns_to_drop)

In [26]:
# Impute missing values for `xlogp` with the median (since it's a numerical feature)
pubchem_df_cleaned['xlogp'] = pubchem_df_cleaned['xlogp'].fillna(pubchem_df_cleaned['xlogp'].median())

In [32]:
# Check the cleaned dataset for missing values
print(pubchem_df_cleaned.isnull().sum())

print(pubchem_df_cleaned.dtypes)


cid                       0
cmpdname                  0
mw                        0
mf                        0
polararea                 0
complexity                0
xlogp                     0
heavycnt                  0
hbonddonor                0
hbondacc                  0
rotbonds                  0
inchi                     0
isosmiles                 0
canonicalsmiles           0
inchikey                  0
exactmass                 0
monoisotopicmass          0
charge                    0
covalentunitcnt           0
isotopeatomcnt            0
totalatomstereocnt        0
definedatomstereocnt      0
undefinedatomstereocnt    0
totalbondstereocnt        0
definedbondstereocnt      0
undefinedbondstereocnt    0
pclidcnt                  0
gpidcnt                   0
gpfamilycnt               0
annothitcnt               0
cidcdate                  0
sidsrcname                0
depcatg                   0
dtype: int64
cid                         int64
cmpdname                   ob

In [36]:
float_columns = ['mw', 'polararea', 'complexity', 'xlogp', 'exactmass', 'monoisotopicmass']
# Round and convert each column to int64
for col in float_columns:
    pubchem_df_cleaned[col] = pubchem_df_cleaned[col].round().astype('int64')

print(pubchem_df_cleaned[float_columns].dtypes)



mw                  int64
polararea           int64
complexity          int64
xlogp               int64
exactmass           int64
monoisotopicmass    int64
dtype: object


In [21]:
if mol is None:
    print("Invalid molecule")
else:
    # Calculate various RDKit descriptors
    mol_weight = Descriptors.MolWt(mol)
    tpsa = Descriptors.TPSA(mol)
    xlogp = Crippen.MolLogP(mol)
    hb_donors = Lipinski.HBondDonorCount(mol)
    hb_acceptors = Lipinski.HBondAcceptorCount(mol)
    rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    heavy_atoms = Descriptors.HeavyAtomCount(mol)
    num_aromatic_rings = Descriptors.NumAromaticRings(mol)
    print(f'Molecular Weight: {mol_weight}')
    print(f'Topological Polar Surface Area (TPSA): {tpsa}')
    print(f'LogP (XLogP): {xlogp}')
    print(f'HBond Donors: {hb_donors}')
    print(f'HBond Acceptors: {hb_acceptors}')
    print(f'Rotatable Bonds: {rotatable_bonds}')
    print(f'Heavy Atom Count: {heavy_atoms}')
    print(f'Number of Aromatic Rings: {num_aromatic_rings}')


NameError: name 'Descriptors' is not defined